# 入门
### Hello World

In [ ]:
import tensorflow as tf
c = tf.constant('hello')
with tf.Session() as sess:
    print(sess.run(c))

### add与+的区别？
实际上最终调用是一个函数，都到了gen_math_ops.add。
tf.add直接调用库方法，+是重载了__add__符号，在tensorflow/python/ops/math_ops.py中定义了_OverrideBinaryOperatorHelper(gen_math_ops.add, "add")，也就是说最终还是调用了tf.add

加载模块tensorflow/python/ops/math_ops.py
```
tensorflow/python/ops/math_ops.py： _OverrideBinaryOperatorHelper(gen_math_ops.add, "add") 
-> tensorflow/python/ops/math_ops.py： clazz_object._override_operator("__%s__" % op_name, binary_op_wrapper) 在binary_op_wrapper中会实际调用gen_math_ops.add
-> tensorflow/python/framework/ops.py： _override_helper(Tensor, operator, func)
-> tensorflow/python/framework/ops.py： setattr(clazz_object, operator, func)
```

* Python 中的setattr有点像Ruby元编程中的动态方法，既可以是属性值，也可以是函数方法。setattr(obj, '__add__', add)也就是动态重载+运算符*

所有重载的运算符在Tensor.OVERLOADABLE_OPERATORS:中进行了定义。

参考：
https://stackoverflow.com/questions/37900780/in-tensorflow-what-is-the-difference-between-tf-add-and-operator

In [20]:
import tensorflow as tf
a = tf.constant(1)
b = tf.constant(2)
with tf.Session() as sess:
    print(sess.run(a+b))
with tf.Session() as sess:
    print(sess.run(tf.add(a,b)))

3
3


### 一个简单的线性模型

In [32]:
import tensorflow as tf
import numpy as np

x_data = np.float16(np.random.rand(100) * 10) # 随机输入
y_data = x_data * 3.2 + 5.1

# 构造一个线性模型
b = tf.Variable(tf.zeros([1]))
W = tf.Variable(tf.zeros([1]))
y = x_data * W + b

# 优化器的rate跟什么相关？直接取决于损失函数 。比如tf.reduce_mean(tf.square(y - y_data))对应0.01，而tf.reduce_sum(tf.pow(y_data - y, 2))对应0.0001
# 平方差，tf.reduce_mean对应平均值
loss = tf.reduce_mean(tf.square(y - y_data))
# 二次方幂差，tf.reduce_sum求和
#loss = tf.reduce_sum(tf.pow(y_data - y, 2)) 
# rate看上面用哪个而定
train = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

# 初始化变量
init = tf.global_variables_initializer()

# 启动图 (graph)
sess = tf.Session()
sess.run(init)

# 拟合平面
for step in range(0, 3000):
    sess.run(train)
    if step % 100 == 0:
        print(step, sess.run(loss), sess.run(W), sess.run(b))

0 41.2291 [ 2.87978077] [ 0.44146872]
100 1.903 [ 3.61912918] [ 2.22529006]
200 0.77168 [ 3.46652532] [ 3.27014828]
300 0.312936 [ 3.36934972] [ 3.93549657]
400 0.126919 [ 3.30747008] [ 4.3591795]
500 0.05149 [ 3.26806617] [ 4.62897301]
600 0.0209043 [ 3.24297452] [ 4.80077219]
700 0.00850202 [ 3.22699666] [ 4.91017056]
800 0.00347274 [ 3.21682167] [ 4.97983694]
900 0.00143349 [ 3.21034265] [ 5.02419758]
1000 0.000606607 [ 3.20621705] [ 5.05244541]
1100 0.000271311 [ 3.20358992] [ 5.07043266]
1200 0.000135351 [ 3.20191717] [ 5.08188581]
1300 8.02121e-05 [ 3.20085192] [ 5.08918047]
1400 5.78552e-05 [ 3.20017362] [ 5.09382439]
1500 4.87819e-05 [ 3.19974136] [ 5.09678316]
1600 4.51038e-05 [ 3.19946623] [ 5.09866762]
1700 4.36151e-05 [ 3.19929123] [ 5.09986591]
1800 4.30103e-05 [ 3.19917941] [ 5.10063028]
1900 4.27641e-05 [ 3.1991086] [ 5.10111618]
2000 4.2667e-05 [ 3.1990633] [ 5.10142565]
2100 4.26256e-05 [ 3.19903445] [ 5.10162258]
2200 4.2608e-05 [ 3.19901657] [ 5.10174561]
2300 4.2603